![mmeda1.png](mmeda1.png)

In [1]:
! tar -xvf ../images.tar

x images/
x images/31183420
x images/25659436
x images/35154351
x images/27420594
x images/6177231
x images/25358901
x images/20574409
x images/837890
x images/15800037
x images/694516
x images/34914961
x images/17182878
x images/17155841
x images/805271
x images/2040753
x images/25878327
x images/15731388
x images/31181724
x images/15716956
x images/18054236
x images/21455562
x images/11455525
x images/32332938
x images/17262776
x images/29325994
x images/13546400
x images/10596592
x images/34713325
x images/29274947
x images/8584482
x images/23491876
x images/16238433
x images/27842455
x images/3155836
x images/29502117
x images/17338374
x images/26090434
x images/500797
x images/6534215
x images/2439888
x images/7739043
x images/868087
x images/18363841
x images/47766
x images/32671419
x images/13735686
x images/821437
x images/147845
x images/43459
x images/23247947
x images/13502110
x images/19144990
x images/16136439
x images/13618377
x images/25757498
x images/1459982
x images/1

x images/17567452
x images/21849684
x images/13581870
x images/29064097
x images/17841042
x images/20959438
x images/34931974
x images/12106351
x images/7222210
x images/35831214
x images/765539
x images/25848647
x images/9439768
x images/16370256
x images/26037638
x images/831874
x images/30530504
x images/22883569
x images/17074229
x images/30671871
x images/25485041
x images/19293913
x images/17277796
x images/632886
x images/17401075
x images/1846917
x images/17333891
x images/30359111
x images/18893787
x images/13230263
x images/29078428
x images/16631206
x images/17568883
x images/2741937
x images/25897855
x images/29621274
x images/30121301
x images/30094338
x images/923472
x images/420031
x images/26346424
x images/15816693
x images/21524238
x images/20881071
x images/13375134
x images/18465982
x images/10252870
x images/13488906
x images/10412948
x images/1707945
x images/18110947
x images/678362
x images/16059619
x images/28504877
x images/13163450
x images/34235301
x images/

x images/8128563
x images/23018850
x images/1142779
x images/6569571
x images/17211573
x images/16178359
x images/25447121
x images/1823986
x images/15727228
x images/16065750
x images/17857477
x images/18109407
x images/17340539
x images/17886045
x images/22365731
x images/490472
x images/17701555
x images/25897850
x images/12020511
x images/10880366
x images/64481
x images/1368
x images/23767137
x images/36349794
x images/196671
x images/17333894
x images/29855394
x images/26535163
x images/22892097
x images/20921108
x images/22882846
x images/18484017
x images/13503480
x images/18752314
x images/24489718
x images/18719534
x images/21405864
x images/11405479
x images/827227
x images/6123462
x images/18307764
x images/25554231
x images/34624724
x images/22383541
x images/8429305
x images/173907
x images/26201145
x images/22137738
x images/6719823
x images/25755153
x images/13353034
x images/32308075
x images/645245
x images/13547434
x images/8549988
x images/21412232
x images/26207963

In [2]:
# import tarfile
# my_tar = tarfile.open('../dataset/images.tar')
# my_tar.extractall('../dataset/images/') # specify which folder to extract to
# my_tar.close()

In [3]:
import torchvision.models as models
import torch
from torchvision import datasets, transforms
from PIL import Image
import os
import numpy as np

In [4]:
import torch.nn as nn
import torch.functional as F

#### Data Loading

In [5]:
# data_dir = '../dataset/images/images/'
data_dir = "./images/"
input_size = 224

In [6]:
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_files = os.listdir(data_dir)
# image_datasets = datasets.ImageFolder(data_dir, data_transforms)
# dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, shuffle=True, num_workers=4)

In [7]:
image_tensor = Image.open(data_dir + image_files[0])
image_tensor = image_tensor.convert('RGB')
image_tensor = data_transforms(image_tensor)
image_tensor = torch.unsqueeze(image_tensor, 0)
error_files = []
count = 0
for image_file in image_files[1:]:
    try:
        img = Image.open(data_dir + image_file)
        img = img.convert('RGB')
        img_t = data_transforms(img)
        img_t = torch.unsqueeze(img_t, 0)
        image_tensor = torch.cat((image_tensor, img_t), 0)
    except:
        error_files.append(image_file)
    count += 1
    if count % 100 == 0:
        print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [8]:
image_tensor.size()

torch.Size([5000, 3, 224, 224])

In [9]:
# torch.save(image_tensor, "image_tensor.pkl")

In [10]:
X1_file ="../dataset/word2vec_emb_tensor.pkl"
# X1_file = "./word2vec_emb_tensor.pkl"
X1 = torch.load(X1_file)
print(X1.size())

torch.Size([5000, 100])


In [11]:
X1_t = X1.transpose(0, 1)

In [12]:
# Convolution Encoder
class ConvEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(ConvEncoder, self).__init__()
        #Encoder
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.lin1 = nn.Linear(12544, 512)
        self.lin2 = nn.Linear(512, embedding_dim)


    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
#         print(x.size())
        x = self.pool(x)
        x = nn.ReLU()(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, start_dim=1)
#         print(x.size())
        x = nn.ReLU()(self.lin1(x))
        x = nn.ReLU()(self.lin2(x))
#         print(x.size())
        return x


In [13]:
# Encoder for real numbered matrix
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super().__init__()
        self.enc_linear1 = nn.Linear(input_dim, input_dim//2)
        self.enc_linear2 = nn.Linear(input_dim//2, embedding_dim)
        self.emb = None
        
    def forward(self, x):
        x = self.enc_linear1(x)
        x = torch.relu(x)
        x = self.enc_linear2(x)
        self.emb = x # return embedding from encoder
        return x # use x for training
    

In [14]:
# Convolution Decoder
class ConvDecoder(nn.Module):
    def __init__(self, embedding_dim):
        super(ConvDecoder, self).__init__()
        #Decoder
        self.lin1 = nn.Linear(embedding_dim, 512)
        self.lin2 = nn.Linear(512, 12544)
        self.deconv1 = nn.ConvTranspose2d(1, 16, 3, stride=2, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(16, 3, 3, stride=1, padding = 2, output_padding=0)
        


    def forward(self, x):
        x = nn.ReLU()(self.lin1(x))
        x = nn.ReLU()(self.lin2(x)).view(-1, 1, 112, 112)
        # print(x.size())
        x = nn.ReLU()(self.deconv1(x))
        x = self.deconv2(x)
        # print(x.size())
        return x


In [15]:
class Arch(nn.Module):
    def __init__(self, embedding_dim, book_size, word2vec_size):
        super().__init__()
        self.conv_encoder = ConvEncoder(embedding_dim)
        self.encoder_row = Encoder(word2vec_size, embedding_dim)
        self.encoder_col = Encoder(book_size, embedding_dim)
        self.conv_decoder = ConvDecoder(embedding_dim)
        self.intermediate_lin1 = nn.Linear(embedding_dim * 2, embedding_dim)
        
    def forward(self, matrices): # replace with one cell recon
        # currently 3 inputs - [image batch, word vec row batch, word vec col batch]
        # ensure the entity of interest is the row entity in m eg: books x images => books are entity of interest
        m_conv_encoded = self.conv_encoder(matrices[0])
        m_row_encoded = self.encoder_row(matrices[1])
        m_col_encoded = self.encoder_col(matrices[2])
        m_row_cat = torch.cat((m_conv_encoded, m_row_encoded), axis = 1)
        m_row_emb = self.intermediate_lin1(m_row_cat)
        X0_prime = self.conv_decoder(m_row_emb)
        
        X1_prime = torch.mm(m_row_emb, m_col_encoded.transpose(0, 1))
        return X0_prime, X1_prime, m_row_emb
        


In [16]:
# Hyperparams
epoch_count = 30
convergence_threshold = 1e-3
batch_size = 50
book_size = 5000
word2vec_size = 100

In [17]:
net = Arch(50, book_size, word2vec_size)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)

In [18]:
def test(sample):
    m_conv_encoded = net.conv_encoder(sample[0].unsqueeze(0))
    m_row_encoded = net.encoder_row(sample[1].unsqueeze(0))
    m_row_cat = torch.cat((m_conv_encoded, m_row_encoded), axis = 1)
    m_row_emb = net.intermediate_lin1(m_row_cat)
    return m_row_emb

In [19]:
def get_embeddings():
    # embeddings = []
    embeddings = np.empty((book_size, embedding_dim), float)
    for i in range(0, book_size):
      sample = []
      sample.append(image_tensor[i])
      sample.append(X1[i])
      # embeddings.append(test(sample).numpy())
      embeddings[i] = test(sample).detach().numpy()
      # print(embeddings.shape)
    np.save("embeddings_new_arch.npy", embeddings)
    # return torch.Tensor(embeddings)

In [21]:
def train():
    # training
    prev_losses = []
    for epoch in range(0,epoch_count):
        avg_loss = 0
        counter = 0
        for r_count in range(0, 5000, batch_size):
            matrix0 = (image_tensor[r_count:r_count + batch_size])
            matrix1 = (X1[r_count:r_count + batch_size])
            for c_count in range(0, 100, batch_size):
              counter += 1
              matrix2 = (X1_t[c_count:c_count + batch_size])
              m0_prime, m1_prime, m_row_emb = net.forward([matrix0, matrix1, matrix2])
              loss = criterion(matrix0, m0_prime) + criterion(matrix1[:,c_count: c_count+batch_size], m1_prime)
              avg_loss += loss.item()
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
        per_epoch_loss = avg_loss/counter
        prev_losses.append(per_epoch_loss)
        if epoch % 10 == 0:
            print(f"Average loss for epoch {epoch} = {per_epoch_loss}")
        if  (epoch > 100) and (len(prev_losses) > 0) and (abs(prev_losses[-1] - loss) < convergence_threshold):
            print('Convergence!')
            break
        prev_losses.append(loss)
        torch.save(net.state_dict(), f"./model/alternate_arch_epoch{epoch}.pkl")


In [22]:
# from google.colab import drive
# drive.mount('/content/drive')

In [23]:
train()

C:\Users\ankit\anaconda3\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Average loss for epoch 0 = 1.8685067105293274
Average loss for epoch 10 = 1.8570747023820877
Average loss for epoch 20 = 1.856342881321907


In [24]:
embedding_dim=50

In [25]:
get_embeddings()

In [26]:
# torch.save(emb, "embeddings_new_arch.pkl")

In [ ]:
from google.colab import files
files.download('embeddings_new_arch.npy')

In [27]:
np.load("embeddings_new_arch.npy").shape

(5000, 50)